In [1]:
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

In [2]:
API_URL = "http://localhost:8000"
EXPERIMENT_NAME = "ad_campaign_test"
ARMS = ["Ad_Creative_A", "Ad_Creative_B", "Ad_Creative_C"]
TRUE_CONVERSION_RATES = [0.05, 0.08, 0.12]  # True CTR for each ad
N_ITERATIONS = 1000

In [3]:
def create_experiment(algorithm="epsilon_greedy", epsilon=0.1):
    response = requests.post(
        f"{API_URL}/experiments",
        json={
            "experiment_name": EXPERIMENT_NAME,
            "arms": ARMS,
            "algorithm": algorithm,
            "epsilon": epsilon
        }
    )
    return response.json()

In [4]:
def simulate_interaction():
    # Select arm
    select_response = requests.post(
        f"{API_URL}/select",
        json={"experiment_name": EXPERIMENT_NAME}
    )
    selection = select_response.json()
    
    # Simulate conversion (Bernoulli reward)
    arm_idx = selection["arm_index"]
    conversion_rate = TRUE_CONVERSION_RATES[arm_idx]
    reward = float(np.random.random() < conversion_rate)
    
    # Update reward
    requests.post(
        f"{API_URL}/update",
        json={
            "experiment_name": EXPERIMENT_NAME,
            "arm_index": arm_idx,
            "reward": reward
        }
    )
    
    return arm_idx, reward

In [6]:
print("Creating experiment...")
create_experiment()

Creating experiment...


{'message': "Experiment 'ad_campaign_test' created successfully"}

In [7]:
print(f"Running {N_ITERATIONS} simulations...")
results = []

Running 1000 simulations...


In [8]:
for i in range(N_ITERATIONS):
    arm, reward = simulate_interaction()
    
    # Get stats every 50 iterations
    if (i + 1) % 50 == 0:
        stats_response = requests.get(
            f"{API_URL}/experiments/{EXPERIMENT_NAME}/stats"
        )
        stats = stats_response.json()
        results.append({
            "iteration": i + 1,
            **stats
        })
        print(f"Iteration {i+1}: Avg Reward = {stats['average_reward']:.4f}")

Iteration 50: Avg Reward = 0.0600
Iteration 100: Avg Reward = 0.0700
Iteration 150: Avg Reward = 0.0733
Iteration 200: Avg Reward = 0.0700
Iteration 250: Avg Reward = 0.0800
Iteration 300: Avg Reward = 0.0900
Iteration 350: Avg Reward = 0.0829
Iteration 400: Avg Reward = 0.0800
Iteration 450: Avg Reward = 0.0778
Iteration 500: Avg Reward = 0.0860
Iteration 550: Avg Reward = 0.0836
Iteration 600: Avg Reward = 0.0817
Iteration 650: Avg Reward = 0.0862
Iteration 700: Avg Reward = 0.0814
Iteration 750: Avg Reward = 0.0813
Iteration 800: Avg Reward = 0.0800
Iteration 850: Avg Reward = 0.0812
Iteration 900: Avg Reward = 0.0811
Iteration 950: Avg Reward = 0.0811
Iteration 1000: Avg Reward = 0.0820


In [9]:
df = pd.DataFrame(results)

In [10]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Average Reward Over Time',
        'Arm Selection Distribution',
        'Arm Value Estimates',
        'Cumulative Pulls per Arm'
    )
)

In [14]:
fig.add_trace(
    go.Scatter(x=df['iteration'], y=df['average_reward'], name='Avg Reward'),
    row=1, col=1
)
fig.add_hline(y=max(TRUE_CONVERSION_RATES), line_dash="dash", 
              annotation_text="Optimal", row=1, col=1)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'Avg Reward',
              'type': 'scatter',
              'x': array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
                           650,  700,  750,  800,  850,  900,  950, 1000], dtype=int64),
              'xaxis': 'x',
              'y': array([0.06      , 0.07      , 0.07333333, 0.07      , 0.08      , 0.09      ,
                          0.08285714, 0.08      , 0.07777778, 0.086     , 0.08363636, 0.08166667,
                          0.08615385, 0.08142857, 0.08133333, 0.08      , 0.08117647, 0.08111111,
                          0.08105263, 0.082     ]),
              'yaxis': 'y'},
             {'name': 'Pulls',
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x2',
              'y': [31, 816, 153],
              'yaxis': 'y2'},
             {'name': 'Estimated Value',
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x3',
              'y': [0.06451612903225806, 0.07965686274509806, 0.09803921568627458],
              'yaxis': 'y3'},
             {'marker': {'color': 'red'},
              'name': 'True Value',
              'opacity': 0.5,
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x3',
              'y': [0.05, 0.08, 0.12],
              'yaxis': 'y3'},
             {'name': 'Avg Reward',
              'type': 'scatter',
              'x': array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
                           650,  700,  750,  800,  850,  900,  950, 1000], dtype=int64),
              'xaxis': 'x',
              'y': array([0.06      , 0.07      , 0.07333333, 0.07      , 0.08      , 0.09      ,
                          0.08285714, 0.08      , 0.07777778, 0.086     , 0.08363636, 0.08166667,
                          0.08615385, 0.08142857, 0.08133333, 0.08      , 0.08117647, 0.08111111,
                          0.08105263, 0.082     ]),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Average Reward Over Time',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Selection Distribution',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Value Estimates',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Cumulative Pulls per Arm',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
           

In [12]:
final_stats = results[-1]
fig.add_trace(
    go.Bar(x=ARMS, y=final_stats['arm_counts'], name='Pulls'),
    row=1, col=2
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'Avg Reward',
              'type': 'scatter',
              'x': array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
                           650,  700,  750,  800,  850,  900,  950, 1000], dtype=int64),
              'xaxis': 'x',
              'y': array([0.06      , 0.07      , 0.07333333, 0.07      , 0.08      , 0.09      ,
                          0.08285714, 0.08      , 0.07777778, 0.086     , 0.08363636, 0.08166667,
                          0.08615385, 0.08142857, 0.08133333, 0.08      , 0.08117647, 0.08111111,
                          0.08105263, 0.082     ]),
              'yaxis': 'y'},
             {'name': 'Pulls',
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x2',
              'y': [31, 816, 153],
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Average Reward Over Time',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Selection Distribution',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Value Estimates',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Cumulative Pulls per Arm',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'showarrow': False,
                                'text': 'Optimal',
                                'x': 1,
                                'xanchor': 'right',
                                'xref': 'x domain',
                                'y': 0.12,
                                'yanchor': 'bottom',
                                'yref': 'y'}],
               'shapes': [{'line': {'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 0.12,
                           'y1': 0.12,
                           'yref': 'y'}],
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.45]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.55, 1.0]},
               'xaxis3': {'anchor': 'y3', 'domain': [0.0, 0.45]},
               'xaxis4': {'anchor': 'y4', 'domain': [0.55, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.625, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.625, 1.0]},
               'yaxis3': {'anchor': 'x3', 'domain': [0.0, 0.375]},
               'yaxis4': {'anc

In [13]:
fig.add_trace(
    go.Bar(x=ARMS, y=final_stats['arm_values'], name='Estimated Value'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=ARMS, y=TRUE_CONVERSION_RATES, name='True Value', 
           marker_color='red', opacity=0.5),
    row=2, col=1
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'Avg Reward',
              'type': 'scatter',
              'x': array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
                           650,  700,  750,  800,  850,  900,  950, 1000], dtype=int64),
              'xaxis': 'x',
              'y': array([0.06      , 0.07      , 0.07333333, 0.07      , 0.08      , 0.09      ,
                          0.08285714, 0.08      , 0.07777778, 0.086     , 0.08363636, 0.08166667,
                          0.08615385, 0.08142857, 0.08133333, 0.08      , 0.08117647, 0.08111111,
                          0.08105263, 0.082     ]),
              'yaxis': 'y'},
             {'name': 'Pulls',
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x2',
              'y': [31, 816, 153],
              'yaxis': 'y2'},
             {'name': 'Estimated Value',
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x3',
              'y': [0.06451612903225806, 0.07965686274509806, 0.09803921568627458],
              'yaxis': 'y3'},
             {'marker': {'color': 'red'},
              'name': 'True Value',
              'opacity': 0.5,
              'type': 'bar',
              'x': [Ad_Creative_A, Ad_Creative_B, Ad_Creative_C],
              'xaxis': 'x3',
              'y': [0.05, 0.08, 0.12],
              'yaxis': 'y3'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Average Reward Over Time',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Selection Distribution',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Arm Value Estimates',
                                'x': 0.225,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Cumulative Pulls per Arm',
                                'x': 0.775,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.375,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'showarrow': False,
                                'text': 'Optimal',
                                'x': 1,
                                'xanchor': 'right',
                                'xref': 'x domain',
                                'y': 0.12,
                                'yanchor': 'bottom',
                                'yref': 'y'}],
               'shapes': [{'line': {'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 0.12,
                          

In [ ]:
for i, arm in enumerate(ARMS):
    pulls = [r['arm_counts'][i] for r in results]
    fig.add_trace(
        go.Scatter(x=df['iteration'], y=pulls, name=arm),
        row=2, col=2
    )

fig.update_layout(height=800, showlegend=True, title_text="MAB Experiment Results")
fig.show()